In [ ]:
import tensorflow as tf
import keras as K
import numpy as np

In [ ]:
# Load original model
original_model = K.models.load_model('models/variationCNN.keras')  # Replace with your model file
#original_model = tf.keras.models.load_model('CNNModel/BaseCNN.keras')  # Replace with your model file
#original_model = tf.keras.models.load_model('CNNModel/simpleCNN.keras')  # Replace with your model file
#original_model = tf.keras.models.load_model('CNNModel/StrongCNN.keras')  # Replace with your model file
original_model.trainable = False  # Freeze weights
loaded_atn_model = K.models.load_model('models/acgan-100epochs.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNInputDiversityhighTransformation.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNInputDiversityHighTransform_noflip_atn2.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNInputDiversity.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNInputDiversity05_atn2.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNInputDiversity07_atn2.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNMultuCNN_Vote_ATN2.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNMultuScale_Vote_ATN2.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNMultModel_ensemble_ATN2.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNbase2_120.keras')
#loaded_atn_model = tf.keras.models.load_model('ATN_model/ATNbase2_120_001.keras')

In [3]:
def get_second_highest_target(y_pred_original):
    # Sort predictions to find the highest and second-highest indices
    top_2_indices = tf.argsort(y_pred_original, direction='DESCENDING', axis=1)[:, :2]
    second_highest_indices = top_2_indices[:, 1]

    # Create one-hot encoding for the second-highest class
    y_target = tf.one_hot(second_highest_indices, depth=y_pred_original.shape[1])
    return y_target


In [ ]:
# Load MNIST datasetatn
mnist_train, mnist_test = tfds.load('mnist', split=['train', 'test'], as_supervised=True)

# Function to create adversarial target labels
#def create_adversarial_target(label):
#    # Example: Shift each label by 1 (cyclic mapping)
#    return (label + 1) % 10

# Preprocessing function for training the ATN
def preprocess_image_atn(image, label):
    # Do NOT normalize the images; keep them in the range [0, 255]
    image = tf.cast(image, tf.float32)  # Ensure float32 for compatibility with loss functions
    label_one_hot = tf.one_hot(label, depth=10)  # One-hot encode the original label
    
    # Return only the raw image and the original label
    return image, label_one_hot

# Apply preprocessing to the dataset
mnist_train_atn = mnist_train.map(preprocess_image_atn)
mnist_test_atn = mnist_test.map(preprocess_image_atn)

# Batch and prefetch the dataset
batch_size = 32
mnist_train_atn = mnist_train_atn.batch(batch_size).prefetch(tf.data.AUTOTUNE)
mnist_test_atn = mnist_test_atn.batch(batch_size).prefetch(tf.data.AUTOTUNE)

print("Dataset loaded without normalization.")


In [ ]:
import matplotlib.pyplot as plt

# Function to evaluate and compare results
def evaluate_atn(original_model, atn, dataset):
    for x_batch, y_original_batch in dataset.take(2):  # Take one batch (two elements per batch in the dataset)
        # Generate transformed images
        x_transformed = atn.predict(x_batch)

        # Get predictions for the original and transformed images
        y_pred_original = tf.argmax(original_model.predict(x_batch), axis=1)
        y_pred_transformed = tf.argmax(original_model.predict(x_transformed), axis=1)

        # Get true labels
        y_true = tf.argmax(y_original_batch, axis=1)

        # Print results for comparison
        print("True Labels:", y_true.numpy())
        print("Original Predictions:", y_pred_original.numpy())
        print("Transformed Predictions:", y_pred_transformed.numpy())

        # Display images
        #display_images(x_batch, x_transformed, y_true, y_pred_original, y_pred_transformed, original_model.predict(x_batch))
        # Find indices where original predictions are correct and transformed predictions differ
        correct_original = y_pred_original == y_true
        incorrect_transformed = y_pred_transformed != y_true
        interesting_indices = tf.where(correct_original & incorrect_transformed)[:, 0]  # Indices of interest

        if len(interesting_indices) == 0:
            print("No interesting cases in this batch.")
            continue

        # Filter the relevant data
        filtered_x_batch = tf.gather(x_batch, interesting_indices)
        filtered_x_transformed = tf.gather(x_transformed, interesting_indices)
        filtered_y_true = tf.gather(y_true, interesting_indices)
        filtered_y_pred_original = tf.gather(y_pred_original, interesting_indices)
        filtered_y_pred_transformed = tf.gather(y_pred_transformed, interesting_indices)

        # Display images and predictions
        display_images(
            filtered_x_batch,
            filtered_x_transformed,
            filtered_y_true,
            filtered_y_pred_original,
            filtered_y_pred_transformed,
            original_model.predict(filtered_x_batch)
        )
# Function to display images
def display_images(original_images, transformed_images, y_true, y_pred_original, y_pred_transformed, original_pred):
    num_images = min(10, len(original_images))  # Display up to 10 images
    plt.figure(figsize=(15, 5))

    for i in range(num_images):
        
        # Original image
        plt.subplot(2, num_images, i + 1)
        plt.imshow(tf.squeeze(original_images[i]), cmap='gray')  # Remove channel for grayscale display
        plt.title(f"Label: {y_true[i].numpy()}, \n Pred: {y_pred_original[i].numpy()}")
        plt.axis('off')

        # Transformed image
        second_highest_target = get_second_highest_target(original_pred[i: i + 1])[0]  # Compute target dynamically
        second_highest_class = tf.argmax(second_highest_target).numpy()

        plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(tf.squeeze(transformed_images[i]), cmap='gray')
        plt.title(f"Target: {second_highest_class}, \n Pred: {y_pred_transformed[i].numpy()}")
        plt.axis('off')

    plt.show()


# Evaluate the ATN on the test dataset


evaluate_atn(original_model, loaded_atn_model, mnist_test_atn)


In [ ]:
# Function to calculate accuracy
def calculate_accuracy(model, dataset, atn=None):
    correct = 0
    total = 0

    for x_batch, y_original_batch in dataset:  # Only unpack two elements
        if atn:
            # Transform images using ATN
            x_batch = atn.predict(x_batch, verbose=0)  # Suppress verbose output
        # Get predictions from the model
        y_pred = tf.argmax(model.predict(x_batch, verbose=0), axis=1)
        y_true = tf.argmax(y_original_batch, axis=1)  # True labels
        # Count correct predictions
        correct += tf.reduce_sum(tf.cast(y_pred == y_true, tf.float32)).numpy()
        total += x_batch.shape[0]  # Batch size
    
    return correct / total  # Calculate accuracy


# Original accuracy
original_accuracy = calculate_accuracy(original_model, mnist_test_atn, atn=None)
print(f"Accuracy on original images: {original_accuracy:.4f}")

# Transformed accuracy
transformed_accuracy = calculate_accuracy(original_model, mnist_test_atn, atn=loaded_atn_model)
print(f"Accuracy on transformed images: {transformed_accuracy:.4f}")
print(f"Performance of ATN: {1-transformed_accuracy:.4f}")